<a href="https://colab.research.google.com/github/zuzannazak/SentimentAnalysisInPolish/blob/main/Sentiment_analysis_in_Polish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Combining multiple files
**Input:** Csv files relating to 1 profile gathered from https://colab.research.google.com/drive/1P0mf1jhhYLqlDVN2eqC2HjbWeEg-6dmJ#scrollTo=QhTw_07h092i, uploaded to Colab in folder called DataToCombine

**Output:** One CSV file

In [ ]:
import os
import glob
import pandas as pd
os.chdir("/content/DataToCombine")

In [ ]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [ ]:
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

#Sentiment analysis
**Input:** Prepared csv files gathered from https://colab.research.google.com/drive/1P0mf1jhhYLqlDVN2eqC2HjbWeEg-6dmJ#scrollTo=QhTw_07h092i and combined per social media account

**Output:** sentiment assigned to each tweet

In [1]:
pip install -i https://pypi.clarin-pl.eu/ lpmn_client==1.4.1

Looking in indexes: https://pypi.clarin-pl.eu/


In [2]:
 pip show lpmn_client

Name: lpmn-client
Version: 1.4.1
Summary: Client for writing LPMN queries.
Home-page: UNKNOWN
Author: Szymon Ciombor, Mateusz Gniewkowski
Author-email: Szymon.Ciombor@pwr.edu.pl, mateusz.gniewkowski@pwr.edu.pl
License: UNKNOWN
Location: /usr/local/lib/python3.7/dist-packages
Requires: pyyaml, requests
Required-by: 


#Sentiment analysis from csv file


In [12]:
import pandas as pd
import numpy as np

import os

from lpmn_client import download_file, upload_file
from lpmn_client import Task

In [4]:
df = pd.read_csv("/content/SearchTweetPaczkomatyPL('2022-01-12T07_05_13.009Z',).csv")
df['sentiment'] = None
df = df.drop(columns=['Unnamed: 0'])

In [5]:
df.head()

,source,conversation_id,text,created_at,lang,in_reply_to_user_id,reply_settings,author_id,id,referenced_tweets,public_metrics,geo,sentiment
0,Twitter for Android,1475050133461032963,"@PaczkomatyPL Pewnie, że tak 😊 Już wszystko za...",2021-12-26T10:40:33.000Z,pl,2.565559e+09,everyone,52522222,1475053917746696196,"[{'type': 'replied_to', 'id': '147505300392113...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,None
1,Twitter for Android,1475050133461032963,"Pierwszy raz mam taką sytuację, że nie mogę od...",2021-12-26T10:25:31.000Z,pl,NaN,everyone,52522222,1475050133461032963,NaN,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,None
2,Twitter Web App,1474893391817232390,@PaczkomatyPL \nJest jakiś problem po waszej s...,2021-12-26T00:02:41.000Z,pl,2.565559e+09,everyone,2799071567,1474893391817232390,NaN,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,None
3,Twitter for Android,1474623367227617285,@CicheMale @MeteoprognozaPL @ebonit_7 @Doppelm...,2021-12-25T22:04:57.000Z,pl,7.113024e+17,everyone,1376873115091881986,1474863762792493059,"[{'type': 'replied_to', 'id': '147462336722761...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,None
4,Twitter Web App,1474623367227617285,@CicheMale @ebonit_7 @DoppelmayrGmbH @_PistenB...,2021-12-25T11:42:00.000Z,pl,7.113024e+17,everyone,2651114600,1474706994225463301,"[{'type': 'replied_to', 'id': '147462336722761...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,None


In [ ]:
for i, row in df.iterrows():
    unique_id = i
    val=(df['text'].values[i])
    with open('plik.txt', 'w') as f:
      f.write(val)
    file_id = upload_file("/content/plik.txt")
    t = Task('any2txt|wcrft2|wsd|ccl_emo({"lang":"polish"})|ccl_emo_stats({"lang":"polish", "split_paragraphs": false}) ')
    output_file_id = t.run(file_id)
    downloaded = download_file(output_file_id, "./out")

    try:
      df_sentiment=pd.read_csv(downloaded, compression='zip', header=0, sep=';')
      df_sentiment = df_sentiment.replace("None", 0)
      df_sentiment = df_sentiment.replace("amb", 0)
      df_sentiment["Polarity"] = pd.to_numeric(df_sentiment["Polarity"])
      column_sum = df_sentiment["Polarity"].sum()
      df['sentiment'].values[0]= column_sum
      print(df['text'].values[i] + ": " + str(column_sum))
    except:
      df['sentiment'].values[i]= None
      print(df['text'].values[i] + ": no sentiment for value " + str(i) )


@PaczkomatyPL Pewnie, że tak 😊 Już wszystko załatwione z Waszym konsultantem! Dogadać się można zawsze i szybko ❤: -1
Pierwszy raz mam taką sytuację, że nie mogę odebrać paczki z @PaczkomatyPL ,bo paczkomat ... zamarzł 😆❄🌨 I nie otwiera się 😭😭: 0
@PaczkomatyPL 
Jest jakiś problem po waszej stronie?
Nie można doładować konta.
Otrzymuję komunikat:
Upsss
Wystąpił nieoczekiwany błąd. Proszę spróbować później.: 0
@CicheMale @MeteoprognozaPL @ebonit_7 @DoppelmayrGmbH @_PistenBully_ @PaczkomatyPL @bp_Polska @festiwalgorski @ewabilanstoch @PhilippLehmkuhl @OnyxBoox Wzajemnie...Spokojnych Świąt 🌲🌟 https://t.co/vMYFv050EN: 0
@CicheMale @ebonit_7 @DoppelmayrGmbH @_PistenBully_ @PaczkomatyPL @bp_Polska @festiwalgorski @ewabilanstoch @PhilippLehmkuhl @OnyxBoox Wesołych świąt! Bo śniegu chyba nie brakuje:): -1
RT @PrezesObywatel: @MariuszMorda @BartoszKowara U mnie tak samo. @PaczkomatyPL robią dobra robotę. Wszystko w ciągu 1 czy 2 dni.: 0
@MariuszMorda @BartoszKowara U mnie tak samo. @PaczkomatyPL

In [9]:
df_sentiment

,Lemma,Syn_id,Polarity,Emotion_names,Emotion_valuations,Units in synset
0,korzystać,1377,0,NaN,NaN,czerpać_korzyści.1(38:cpos) dobrze_wychodzić.1...
1,konkurencja,1340,0,NaN,NaN,kompetycja.1(8:zdarz) wyścig.2(8:zdarz) współz...


In [10]:
df

,source,conversation_id,text,created_at,lang,in_reply_to_user_id,reply_settings,author_id,id,referenced_tweets,public_metrics,geo,sentiment
0,Twitter for Android,1475050133461032963,"@PaczkomatyPL Pewnie, że tak 😊 Już wszystko za...",2021-12-26T10:40:33.000Z,pl,2.565559e+09,everyone,52522222,1475053917746696196,"[{'type': 'replied_to', 'id': '147505300392113...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,0
1,Twitter for Android,1475050133461032963,"Pierwszy raz mam taką sytuację, że nie mogę od...",2021-12-26T10:25:31.000Z,pl,NaN,everyone,52522222,1475050133461032963,NaN,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,0
2,Twitter Web App,1474893391817232390,@PaczkomatyPL \nJest jakiś problem po waszej s...,2021-12-26T00:02:41.000Z,pl,2.565559e+09,everyone,2799071567,1474893391817232390,NaN,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,0
3,Twitter for Android,1474623367227617285,@CicheMale @MeteoprognozaPL @ebonit_7 @Doppelm...,2021-12-25T22:04:57.000Z,pl,7.113024e+17,everyone,1376873115091881986,1474863762792493059,"[{'type': 'replied_to', 'id': '147462336722761...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,0
4,Twitter Web App,1474623367227617285,@CicheMale @ebonit_7 @DoppelmayrGmbH @_PistenB...,2021-12-25T11:42:00.000Z,pl,7.113024e+17,everyone,2651114600,1474706994225463301,"[{'type': 'replied_to', 'id': '147462336722761...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,Twitter for Android,1479235375117778946,@OLogistyczny @InPostPL1 Chyba znowu oznaczyli...,2022-01-06T23:43:47.000Z,pl,1.382288e+18,everyone,1227343711278944257,1479237288097943555,"[{'type': 'replied_to', 'id': '147923537511777...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,0
937,Twitter for Android,1478035206904950788,@PaczkomatyPL @RBrzoska Czyli mam czekać 30 dn...,2022-01-06T18:23:59.000Z,pl,8.590470e+17,everyone,859047002381967361,1479156810233700355,"[{'type': 'replied_to', 'id': '147908802222645...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,0
938,Twitter for iPad,1478833901510475785,@PaczkomatyPL @HERYTlERA A czy w piątek kurier...,2022-01-06T16:24:41.000Z,pl,2.565559e+09,everyone,291712315,1479126786923548672,"[{'type': 'replied_to', 'id': '147883390151047...","{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,0
939,Twitter Web App,1478035206904950788,@PaczkomatyPL 2/2 do tego reklamacja przez str...,2022-01-06T13:50:39.000Z,pl,8.590470e+17,everyone,859047002381967361,1479088022226452480,"[{'type': 'replied_to', 'id': '147908719495022...","{'retweet_count': 0, 'reply_count': 2, 'like_c...",NaN,0


In [ ]:
for key in accountids:

#Visualisation
Materials:

https://medium.com/@siddhardhan23/data-visualization-in-python-a90ddb706b23

https://towardsdatascience.com/5-python-gui-frameworks-to-create-desktop-web-and-even-mobile-apps-c25f1bcfb561



@PaczkomatyPL Pewnie, że tak 😊 Już wszystko załatwione z Waszym konsultantem! Dogadać się można zawsze i szybko ❤: -1
Pierwszy raz mam taką sytuację, że nie mogę odebrać paczki z @PaczkomatyPL ,bo paczkomat ... zamarzł 😆❄🌨 I nie otwiera się 😭😭: 0
@PaczkomatyPL 
Jest jakiś problem po waszej stronie?
Nie można doładować konta.
Otrzymuję komunikat:
Upsss
Wystąpił nieoczekiwany błąd. Proszę spróbować później.: 0
@CicheMale @MeteoprognozaPL @ebonit_7 @DoppelmayrGmbH @_PistenBully_ @PaczkomatyPL @bp_Polska @festiwalgorski @ewabilanstoch @PhilippLehmkuhl @OnyxBoox Wzajemnie...Spokojnych Świąt 🌲🌟 https://t.co/vMYFv050EN: 0
@CicheMale @ebonit_7 @DoppelmayrGmbH @_PistenBully_ @PaczkomatyPL @bp_Polska @festiwalgorski @ewabilanstoch @PhilippLehmkuhl @OnyxBoox Wesołych świąt! Bo śniegu chyba nie brakuje:): -1
RT @PrezesObywatel: @MariuszMorda @BartoszKowara U mnie tak samo. @PaczkomatyPL robią dobra robotę. Wszystko w ciągu 1 czy 2 dni.: 0
@MariuszMorda @BartoszKowara U mnie tak samo. @PaczkomatyPL robią dobra robotę. Wszystko w ciągu 1 czy 2 dni.: 0
@Favoreq @PaczkomatyPL Ale dziewczyny maja dzięki Tobie @Favoreq cudowne święta! Dziś Pani Ela idzie do znajomych, jutro z Tobą ♥️ namawiałam ją aby przyszła do nas na wigilie, ale się nie zgodziła. Tatiana, cudownie, ze jesteś ♥️♥️♥️♥️: 0
@OkunAdam @AdamDee_ASD @MariuszMorda @PocztaPolska @PaczkomatyPL Nie korzystam z konkurencji https://t.co/HaHjmyY5RK: 0
Nikt nie zrobił więcej dla zdrowia publicznego w czasie pandemii niż @PaczkomatyPL . Szczególnie w czasie świąt.

Fakt nie opinia.: 0
@PaczkomatyPL @MariuszMorda Teraz jeszcze dopracować żeby dochodziły całe i będzie perfecto 👌: 0
@dhl_kurier @AdamDee_ASD @MariuszMorda @PocztaPolska @PaczkomatyPL Nie stać ich... 😉: 0
@MariuszMorda @PaczkomatyPL in post już 2-3 lata temu zostawił konkurencję daleko z tyłu... Mają najlepiej ustawioną logistykę i są tego efekty, dlatego każdy prezent miał na święta 🎅 🎄 @RBrzoska jest Mistrzem!!! 💪👌: 2
@dhl_kurier @AdamDee_ASD @MariuszMorda @PocztaPolska @PaczkomatyPL Odbierałbyś w paczkomacie awizo...: 0
Prezenty nie muszą być pięknie opakowane, żeby były radosne! W tej paczce, którą całą minutę odrywałam od @PaczkomatyPL na mrozie, bo jakimś cudem przymarzła, jest osiem kilo puszek dla Zuzi pani Eli 🐕🎅 Jutro zawiozę 🤗 https://t.co/IIcwuV1McO: 1
@CicheMale @MeteoprognozaPL @ebonit_7 @DoppelmayrGmbH @_PistenBully_ @PaczkomatyPL @bp_Polska @festiwalgorski @ewabilanstoch @PhilippLehmkuhl @OnyxBoox Dziękujemy i dla Was wszystkiego co najlepsze... https://t.co/FtDGbKQxjC: 1
@AdahoPi @n4k1r @PaczkomatyPL Nie naliczy. A przynajmniej mi kiedyś nie naliczyło, jak paczkę, której deadline odbioru upływał w niedzielę o 11 odebrałem w niedzielę o 17.

Ale nie zdziwiłbym się, jakby opróżniali jutro, więc raczej bym się wybrał przed tą 10.

A jak paczka ważna to można ją przedłużyć.: 0
Zgadzam się. @PaczkomatyPL jestem pełen podziwu 👏👏👏 https://t.co/FiUYnJ1kDq: 1
@n4k1r @PaczkomatyPL Olej, ja mam poprawiony czas do 26 😉: 2
@n4k1r @PaczkomatyPL Raczej nie będą opróżniać, ale nie wiem czy i tak ci nie policzą dodatkowej opłaty za opóźnienie i nie otworzy jak jej nie opłacisz.: 1
Ej, jest sprawa. Właśnie mi @PaczkomatyPL przypomnieli w apce, że do jutra do 10 mam odebrać paczkę.

Kurierzy będą jutro opróżniać paczkomaty? Mam się zbierać o 9 i drałować do miasta, czy olać i spokojnie odbiorę po 18?: 1
@dhl_kurier @MariuszMorda @PocztaPolska @PaczkomatyPL Pamiętam.: 0
@AdamDee_ASD @MariuszMorda @PocztaPolska @PaczkomatyPL Poczta miała zamiar wykupić Inpost z całą infrastrukturą paczkomatowa: 0
@PaczkomatyPL Tak! Po Świętach! Gratuluję dobrego samopoczucia. Gdybym wiedział, że tyle czasu potrzebujecie Państwo na dostarczenie paczki z Łodzi do Piotrkowa (40 km) pominąłbym Państwa w łańcuchu dostaw. Dzięki za lekcję!: 0
@MariuszMorda @PocztaPolska powinna zakupić @PaczkomatyPL i zrobić z tym porządek. W głębokim duchu logiki #CieciaRP ☝️😶: -1
@MariuszMorda @PaczkomatyPL  dejcie Panu zniżeczke: 0
@ElzbietaZiober @PaczkomatyPL Nie ma szans. Przy tej pogodzie to jeden dzień jazdy i drugi na powrót.: 2
@PawelBednarek1 @PaczkomatyPL Samo życie, bardzo smutne będą.
W zeszłym roku nie przyjechały, bo nie bylismy jeszcze zaszczepieni, w tym roku miały być święta full-wypas i masz... zapalenie oskrzeli dopadło najmłodszą.
Wpadaj, odległość nie ma znaczenia - @PaczkomatyPL dowodem :): -2
@KarenMillen @PaczkomatyPL Thank you John, I have sent you a DM, just awaiting your reply.: 0
@zawiszowski @PaczkomatyPL To akurat prawda mi też tak przyszedł: 0
@ElzbietaZiober @PaczkomatyPL Smuteczek. Sami zostaliście. Chętnie bym wpadł ale to drugi koniec Polski.: 2
@ktosullivan_uk @PaczkomatyPL Hello there,

I'm glad you told me about this issue. Sorry about this. Please can you drop us a private message and  I'll get this looked into for you. -john: 0
.@PaczkomatyPL jesteście WIELCY!❤️
Wczoraj nadane paczki, dzisiaj dostarczone (ok.400km)
Wnusia zachorowała,nie przyjadą na święta.
Płacz,rozpacz,wszystkie prezenty u nas, dramat...
Liczyłam na cud, cudu nie było - była fantastyczna praca Waszych pracowników. Bardzo dziękuję😍: 1
Still waiting for my delivery which should have been here Monday. The worst courier system in the world InPost are predicting delivery for 23rd December. That would be a Christmas miracle! Appalling service. @KarenMillen @PaczkomatyPL: -2
@zawiszowski @PaczkomatyPL Mi dziś przyszło PS5 ale od MediaMarkt i kuriera DPD :): 0
🎅 W Wigilię uczcijmy gromkim "Alleluja!" wszystkich kurierów oraz menadżerów logistyki InPostu. Wczoraj ostatni prezent, który byłem przekonany że już nie dojdzie, znalazł się w moim paczkomacie... 9h po wysłaniu przez sklep!

@PaczkomatyPL - robicie to zajebiście dobrze 💛 https://t.co/xDUAqBKrTA: 1
RT @PaczkomatyPL: Pierwszy w Polsce system wtórnego obiegu opakowań w e-commerce staje się faktem! Wspólnie z marką @Modivo testowo wdrażam…: 1
@mrsGasky @PaczkomatyPL Paczkomaty zdecydowanie rządzą jeśli chodzi o dostawy 💛: -1
@mrsGasky @PaczkomatyPL Inpost ogarnia jak złoto ✨: 0
Serio, @PaczkomatyPL w tym roku taki kox. Ja widzę paczkomat z okna i dosłownie co chwilę chłopaki podjedzaja. Nawet o chorych godzinach w nocy. 🥰❤️ https://t.co/yymsTHBLpg: 0
@PaczkomatyPL @RBrzoska podobno 98% paczek dociera w 24h. Ta, na którą czekam, nie została nawet odebrana z paczkomatu przez kuriera w takim czasie. Dzięki Paczkomaty! https://t.co/kaClUrmuS1: 0
RT @PaczkomatyPL: Święta Bożego Narodzenia nadchodzą wielkimi krokami! Jeśli świąteczne zakupy wciąż przed Wami, to przypominamy, że wszyst…: 1
Halo @PaczkomatyPL proszę się ogarnąć i oddać paczkę mojej siostrze tak się nie robi wy dranie: -1
Dzięki @PaczkomatyPL ciężki okres świąteczny za nami. Wszystkiego dobrego dla wszystkich 👌 https://t.co/lRAULBvphu: -1
RT @PaczkomatyPL: Już tu jest! Paczkomat #16000 wyposażony w czujnik powietrza stanął w Gorzowie Wielkopolskim - mieście partnerskim progra…: 0
📦 @PaczkomatyPL płacą złotówkę za polubienie 💻 👍
https://t.co/W33M5FRHx5: 4
@PaczkomatyPL Zobaczcie swoje DM bo już zgłaszałem wam o dwóch paczkach. Kolejna paczka niby dwa razy mnie nie było w domu a kurier przez telefon powiedział, że mam się walić. Amazon mi chciał to wynagrodzić, że paczka nie dojdzie i wysłał nową i wyglada jak na zdjęciach.: 0
